In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import pandas as pd
import numpy as np
import datasurfer as ds
from datasurfer.lib_llm.llm_agents import LLMAgent
from datasurfer.lib_objects.docx_object import DOCXObject
from datasurfer.lib_objects.xlsx_object import XLSXObject
from datasurfer.lib_objects.pdf_object import PDFPagesObject


In [15]:
frcs = r'D:\02_Translation\01_Künstliche_Intelligenz_für_Ingenieure\03_KII_Chapter_03\KII_Chapter_03_Review_R1V2.xlsx'
fdst1 = r'D:\02_Translation\01_Künstliche_Intelligenz_für_Ingenieure\03_KII_Chapter_03\KII_Chapter_03_Translation_R2V1.xlsx'
fdst2 = r'D:\02_Translation\01_Künstliche_Intelligenz_für_Ingenieure\03_KII_Chapter_03\KII_Chapter_03_Translation_R2V1.docx'

In [16]:
Linda = LLMAgent('Linda', 'You are a Chinese linguist, you translate German to Chinese.')
Robin = LLMAgent('Robin', 'You are a Chinese linguist, you also know German.')

In [34]:
df_original = ds.AutoObject(frcs)

df = df_original.df[['raw_text', 'translation', 'review']]
df.columns = ['Original', 'Translation', 'Review']

df.Review = df['Review'].fillna(0).astype(int)
df

C:\Users\weiyu\AppData\Local\Temp\ipykernel_5768\2850018993.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Review = df['Review'].fillna(0).astype(int)


,Original,Translation,Review
0,3,三,1
1,Graphensuche,图形搜索,4
2,Dieses Kapitel behandelt die Grundidee von Suc...,本章介绍了搜索算法的基本思想，以图中路径的确定为例，并阐述了所处理算法的共同结构。启发式扩展...,2
3,"A∗-Algorithmus, dessen Wirkungsweise am Beispi...",A* 算法，其工作原理以机器人路径规划为例,2
4,erläutert wird.,将被解释。,2
...,...,...,...
776,92\t3 Graphensuche,图搜索,1
777,Literaturhinweise,文献参考,4
778,Graphensuchalgorithmen sind in vielen Büchern ...,"图搜索算法在许多关于图论的书籍中都有详细的描述，例如[33, 124]。在人工智能领域，这些...",4
779,"Der Dijkstra-Algorithmus, der heute ein Standa...",Dijkstra 算法，今天是确定最短路径的标准方法，首次在[28]中描述，此后在不同的细节...,4


In [31]:
pattern_Linda = 'Translating "{original}" to Chinese, return only the translation, do not include any other words.'
pattern_Robin = '根据德语原文\n"{original}"，\n将以下中文翻译改进到语义通顺并修改翻译的错误：\n"{translation}"\n只返回修改过的句子.'
print(pattern_Robin)


根据德语原文
"{original}"，
将以下中文翻译改进到语义通顺并修改翻译的错误：
"{translation}"
只返回修改过的句子.


In [32]:
def transview_text(original):
    """
    Translates and reviews a given text.

    This function takes an original text and its translation, processes the translation
    using Linda's pattern, and then reviews the translation using Robin's pattern.

    Args:
        original (str): The original text to be translated and reviewed.
        translation (str): The initial translation of the original text.

    Returns:
        list: A list containing the original text and the reviewed translation.
    """
    while 1:

        translation = Linda.told(pattern_Linda.format(original=original), use_cache=False, memory_length=100)
        if 'Instruction' not in translation:
            break

    reviewed = Robin.told(pattern_Robin.format(original=original, translation=translation), use_cache=True, memory_length=200)
    return [original, reviewed] 

In [33]:
out = []
buffer = []
for idx, (original, trans, review) in df.loc[25:].iterrows():
    
    print(f'Processing {idx+1}/{len(df)} ({(idx+1)/len(df)*100:0.2f}%)...')
    
    if review != 2:
        if len(buffer):
            merged = ' '.join(buffer)
            buffer = []
            out.append(transview_text(merged))            

        
    if review == 4:
        out.append(transview_text(original))  
        
    elif review == 2:
        buffer.append(original)
        
    elif review == 1:
        continue
    
    elif review == 3:
        txts = original.split('@')
        
        for txt in txts: 
            out.append(transview_text(txt))
    
    elif review == 0:
        Linda.append_history(pattern_Linda.format(original=original), role='User')
        Linda.append_history(trans)
        Robin.append_history(pattern_Robin.format(original=original, translation=trans), role='User')
        Robin.append_history(trans)
        out.append([original, trans])
    else:
        raise ValueError(f'Invalid review value: {review}')
        
    


Processing 26/781 (3.33%)...
Processing 27/781 (3.46%)...
Processing 28/781 (3.59%)...
Processing 29/781 (3.71%)...
Processing 30/781 (3.84%)...
Processing 31/781 (3.97%)...


BadRequestError: <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Error</title>
</head>
<body>
<pre>Bad Request</pre>
</body>
</html>

In [10]:
raw_text,  translation = zip(*out)

In [11]:
dfout = pd.DataFrame({'Original': raw_text, 'Translation': translation, 'Review': None})

In [12]:
dfout

,Original,Translation,Review
0,13,十三,None
1,Heuristische Verfahren zur Darstellung und zur...,启发式处理不确定性知识的方法。 \n,None
2,Heuristische Verfahren verwenden Kennwerte für...,启发式方法使用指标来表示不确定性或可能性，这些指标用于判断陈述的有效性，并将其组合成指标来衡...,None
3,13.1 Wissensverarbeitung auf der Grundlage der...,13.1 基于证据理论的知识处理,None
4,Die Evidenztheorie wurde von DEMPSTER und SHAF...,Dempster和Shafer开发了证据理论来处理有关未知事物的陈述，这些陈述包含某些正确的...,None
...,...,...,...
227,Literaturhinweise,文献参考\n\n\n\n,None
228,Die Grundlagen der Evidenztheorie wurden von S...,"证据理论的基础是由 SHAFER 于 1976 年奠定的 [110, 111]。[133] ...",None
229,"Das Expertensystem MYCIN, dessen Behandlungsme...",MYCIN 专家系统，其处理不确定性知识的方法为许多其他专家系统应用提供了典范，由 BUCH...,None
230,Die Verwendung grober Mengen (rough sets) ist ...,使用粗糙集是另一种（此处未讨论的）方法，用于通过未知的陈述或事件集合来表示知识。在这种方法中...,None


In [13]:
dfout.to_excel(fdst1, index=False)

In [14]:
obj = ds.DOCXObject(dfout[['Original', 'Translation']], name='Translation')
obj.save_df(fdst2)

ldf is not supported
xls is not supported
d:\13_dev\10_datastructure\datasurfer\datainterface.py:38: UserWarning: Cannot import module "fin_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\10_datastructure\datasurfer\datainterface.py:38: UserWarning: Cannot import module "hdf_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\10_datastructure\datasurfer\datainterface.py:38: UserWarning: Cannot import module "matlab_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\10_datastructure\datasurfer\datainterface.py:38: UserWarning: Cannot import module "pptx_object".
  warnings.warn(f'Cannot import module "{mdlname}".')


<DOCXObject@Translation>